In [1]:
# Cell 1: Import required libraries
import pandas as pd
import numpy as np
import re
import os
from collections import defaultdict
import math
import matplotlib.pyplot as plt

# Make sure plots appear inline
%matplotlib inline


In [1]:
from parse_crystfel_stream import parse_crystfel_stream
from label_asu_indices_cctbx import label_asu_indices_cctbx
from intensity_descriptive_stats import intensity_descriptive_stats
from plotting import plot_raw_intensity_hist, plot_intensity_hist_per_event

# Example usage (adjust file paths and space_group_number as needed):

# 1) Parse
stream_file = "/Users/xiaodong/Desktop/UOX-data/UOX1_sub/centers_xatol_0.01_frameinterval_5_lowess_0.53_shifted_0.5_-0.3/xgandalf_iterations_max_radius_1.8_step_0.5/UOX_0.0_0.0.stream"
df_reflections, df_crystal = parse_crystfel_stream(stream_file, debug=False)

# 2) Label reflections with the ASU indices (no intensity merging!)
#    Choose a space group number that reflects your actual crystal symmetry.
df_asu = label_asu_indices_cctbx(
    df_reflections, 
    df_crystal, 
    space_group_number=23,  # e.g. 19 = P 21 21 21
    anomalous_flag=False,
    debug=True
)

# 3) Inspect or plot
intensity_descriptive_stats(df_asu)
plot_raw_intensity_hist(df_asu)
plot_intensity_hist_per_event(df_asu)



Event='1-1': crystal_sym=Unit cell: (80.4819, 92.6785, 104.864, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='10-1': crystal_sym=Unit cell: (80.6713, 94.1839, 104.689, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='11-1': crystal_sym=Unit cell: (80.613, 94.3351, 106.389, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='12-1': crystal_sym=Unit cell: (80.7831, 94.9588, 103.657, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='13-1': crystal_sym=Unit cell: (80.0994, 95.9264, 103.593, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='14-1': crystal_sym=Unit cell: (81.6571, 94.1577, 105.588, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='18-1': crystal_sym=Unit cell: (80.8506, 94.357, 103.345, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='19-1': crystal_sym=Unit cell: (81.6035, 92.8843, 105.172, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='20-1': crystal_sym=Unit cell: (80.6627, 94.7426, 104.671, 90, 90, 90)
Space group: I 2 2 2 (No. 23)
Event='21-1': crystal_sym=Unit cell: (80

KeyboardInterrupt: 

In [ ]:
# Cell 3: Analyze the distribution of intensities for symmetry-equivalent reflections

def symmetry_key_simple(h, k, l):
    """
    A naive 'symmetry' key: sort absolute values of (h, k, l).
    For real space-group analysis, replace this with a proper
    equivalence operator routine or reciprocal-lattice transform.
    """
    return tuple(sorted([abs(h), abs(k), abs(l)]))

def analyze_symmetry_equivalents(df):
    """
    Group reflections by 'symmetry_key_simple', compute the
    mean intensity, std dev, and a 'coefficient of variation' (CV).
    """
    # Create a key column
    df['sym_key'] = df.apply(lambda row: symmetry_key_simple(row['h'], row['k'], row['l']), axis=1)
    
    # Group by chunk + sym_key so each chunk is handled separately
    grouped = df.groupby(['event','sym_key'])
    
    stats = grouped['I'].agg(['mean','std','count']).reset_index()
    stats.rename(columns={'mean':'I_mean','std':'I_std','count':'N'}, inplace=True)
    
    # Merge to original DataFrame so we can look up each reflection's group stats
    merged = df.merge(stats, on=['event','sym_key'], how='left')
    # Coefficient of variation (avoid division by zero)
    merged['CV_sym'] = merged['I_std'] / (merged['I_mean'].replace(0, np.nan))
    
    return merged

# Usage
df_sym = analyze_symmetry_equivalents(df_reflections)
df_sym.head(10)


KeyError: 'event'

In [ ]:
# Cell 4: Systematic row analysis (example)

def example_forbidden_condition(h, k, l):
    """
    Example rule for orthorhombic-like systems:
    - Only apply if k = 0
    - Then check if (h + l) is even => allowed, else forbidden
    This is just an illustration.
    """
    if k == 0:
        # "Allowed" if (h + l) is even, "forbidden" otherwise
        return ((h + l) % 2 != 0)  # True if forbidden
    else:
        return False  # Not in the systematic row => not considered here

def analyze_systematic_rows(df):
    """
    Demonstrate how to isolate one systematic row and compute a 'forbidden ratio'.
    """
    # We'll define a new column 'is_forbidden'
    df['is_forbidden'] = df.apply(lambda row: example_forbidden_condition(row['h'], row['k'], row['l']), axis=1)
    
    # We only consider those reflections actually in the row k=0 for the ratio
    in_row = df[df['k'] == 0].copy()
    
    # Summation of intensities for "forbidden" vs "allowed"
    # We'll group by chunk_id so we get a per-chunk measure
    def f_ratio(sub):
        forbidden_sum = sub.loc[sub['is_forbidden'], 'I'].sum()
        allowed_sum = sub.loc[~sub['is_forbidden'], 'I'].sum()
        return forbidden_sum / (allowed_sum + 1e-9)
    
    ratio_series = in_row.groupby('chunk_id').apply(f_ratio).reset_index()
    ratio_series.columns = ['chunk_id','forbidden_ratio']
    
    return ratio_series

# Usage
systematic_ratios = analyze_systematic_rows(df_sym)
systematic_ratios.head()


In [ ]:
# Cell 5: Approximate Sayre equation analysis

def build_intensity_dict(df):
    """
    Build a dict: intensity_dict[chunk_id][(h,k,l)] = I
    so we can quickly look up intensities chunk by chunk.
    """
    intensity_dict = defaultdict(dict)
    for idx, row in df.iterrows():
        cid = row['chunk_id']
        hkl = (row['h'], row['k'], row['l'])
        intensity_dict[cid][hkl] = row['I']
    return intensity_dict

def approximate_sayre_intensity(h, k, l, intensity_map, hkl_range=3):
    """
    For a single reflection (h,k,l), approximate I_sayre.
    We'll sum over K in +/- hkl_range.
    """
    sum_val = 0.0
    for hk1 in range(-hkl_range, hkl_range+1):
        for hk2 in range(-hkl_range, hkl_range+1):
            for hk3 in range(-hkl_range, hkl_range+1):
                K = (hk1, hk2, hk3)
                h2, k2, l2 = h - hk1, k - hk2, l - hk3
                if K in intensity_map and (h2, k2, l2) in intensity_map:
                    I_k  = max(intensity_map[K], 0)
                    I_hk = max(intensity_map[(h2, k2, l2)], 0)
                    sum_val += math.sqrt(I_k)*math.sqrt(I_hk)
    return sum_val**2

def analyze_sayre_equation(df, hkl_range=3):
    """
    Compute a 'Sayre misfit' per chunk.
    """
    # Build dictionary for quick lookups
    intensity_dict = build_intensity_dict(df)
    
    results = []
    for chunk_id in intensity_dict.keys():
        # We'll build a list of (I_obs, I_sayre) for each reflection in this chunk
        local_map = intensity_dict[chunk_id]
        misfit_num = 0.0
        misfit_den = 0.0
        
        for (h, k, l), I_obs in local_map.items():
            I_sayre = approximate_sayre_intensity(h, k, l, local_map, hkl_range=hkl_range)
            misfit_num += abs(I_obs - I_sayre)
            misfit_den += abs(I_obs)
        
        # Avoid division by zero
        misfit = misfit_num / (misfit_den + 1e-9)
        results.append({'chunk_id': chunk_id, 'sayre_misfit': misfit})
    
    sayre_df = pd.DataFrame(results)
    return sayre_df

# Example usage:
sayre_metrics = analyze_sayre_equation(df_sym, hkl_range=3)
sayre_metrics.head()


In [ ]:
# Cell 6: Compute Orientation Matrix and Zone-Axis Check from df_cryst

import numpy as np
import pandas as pd

def check_zone_axis(orientation_matrix, threshold=0.8, debug=False):
    """
    Given a 3x3 orientation matrix (each row is a reciprocal lattice vector),
    check if any unit vector is nearly parallel to the beam direction [0, 0, 1].
    Returns (is_zone, dot_value) where dot_value is the highest absolute dot product.
    """
    beam = np.array([0, 0, 1])
    max_dot = 0.0
    for i, vec in enumerate(orientation_matrix):
        norm = np.linalg.norm(vec)
        if norm == 0:
            continue
        unit = vec / norm
        dot = abs(np.dot(unit, beam))
        if debug:
            print(f"   Vector {i+1}: {vec}, unit: {unit}, dot with beam: {dot:.3f}")
        if dot > max_dot:
            max_dot = dot
    is_zone = max_dot >= threshold
    if debug:
        print(f"   --> Maximum dot: {max_dot:.3f} (threshold: {threshold}), is_zone: {is_zone}")
    return is_zone, max_dot

def compute_orientation_fields(df_cryst, debug=False):
    """
    For each row in df_cryst, compute:
      - orientation_matrix as a numpy array from astar, bstar, cstar
      - is_zone_axis (True if any reciprocal vector is nearly parallel to [0,0,1])
      - zone_dot (the highest dot product value)
    The function prints debug messages if requested.
    """
    orientation_list = []
    is_zone_list = []
    zone_dot_list = []
    
    for idx, row in df_cryst.iterrows():
        try:
            # Ensure astar, bstar, cstar are in list form.
            # They may be stored as lists or as strings; if strings, we use eval().
            for col in ['astar', 'bstar', 'cstar']:
                if isinstance(row[col], str):
                    # Caution: using eval() assumes trusted input.
                    row[col] = eval(row[col])
            # Build orientation matrix
            orientation_matrix = np.array([row['astar'], row['bstar'], row['cstar']])
            orientation_list.append(orientation_matrix)
            if debug:
                print(f"\nChunk {row['chunk_id']} orientation matrix:")
                print(orientation_matrix)
            # Check if any vector is nearly along the beam direction [0,0,1]
            is_zone, dot_val = check_zone_axis(orientation_matrix, threshold=0.95, debug=debug)
            is_zone_list.append(is_zone)
            zone_dot_list.append(dot_val if dot_val is not None else np.nan)
        except Exception as e:
            if debug:
                print(f"Error processing chunk {row['chunk_id']}: {e}")
            orientation_list.append(None)
            is_zone_list.append(False)
            zone_dot_list.append(np.nan)
    
    df_cryst['orientation_matrix'] = orientation_list
    df_cryst['is_zone_axis'] = is_zone_list
    df_cryst['zone_dot'] = zone_dot_list
    return df_cryst

# Example usage:
# Assuming df_cryst is the DataFrame with crystal metadata from your stream file.
df_cryst = compute_orientation_fields(df_crystal, debug=True)
print("\nUpdated crystal DataFrame with orientation info:")
print(df_cryst.head())
